# Title
[]()

In [1]:
import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
import time
import re
from itertools import product
import openai

from response_processing import *
from article_processing import create_text_dict_from_folder
import traceback
from file_functions import *
from summary_chain import *

In [2]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set up

In [6]:
chatbot_dict = dict()
simple_summaries_dict = dict()
relevance_dict = dict()
chain_results_dict = dict()
qna_dict = dict()
texts = dict()

In [7]:
# Create text dictionary
folder_path = '../text/2023-06-14 full2' # ** UPDATE REQUIRED**
text_id = 1

encoding='ISO-8859-1'
subset=None

texts[text_id] = create_text_dict_from_folder(folder_path, encoding=encoding, subset=subset)
text_dict = texts[text_id]


Keys for text_dict: dict_keys([1, 2, 3, 4])



# Iteration 1

## prompts

In [10]:
prep_step = [
    "Think about why this might be relevant for the audience in the grand scheme of things.\
    \nIdentify 1 or 2 key concepts from this article that would make interesting or helpful health content. \
    Exclude details that do not add value to the audience.\
    \nBased on the key concepts from the previous steps, extract the key points and statistics to",
]

summarize_task = [
    "summarize for a LinkedIn post.",
    # "Describe the interesting points to your coworker at the water cooler",
    # "Create an Instagram post without hashtags.",
]
edit_task = [
    "\nIf applicable, include a brief description of the research participants, \
    such as age and sex, within the body of the summary. \
    Otherwise, you can skip this step.\
    \nEvaluate whether or not your writing may be confusing or redundant. \
    \nIf so, re-write it so it is clear and concise. Otherwise, keep it the same. \
    \nCreate an intriguing headline to hook the audience.\
    \nReturn your response in this format:\
    \n<headline>\n\n<summary>\
    \nwhere the summary is a publication-ready format.\
    \nDo not label the headline and summary.",
]

system_role = "You are a journalist writing content based on science research articles."
prompts_df = pd.DataFrame(product(prep_step, summarize_task, edit_task), 
    columns=['prep_step', 'summarize_task', 'edit_task'])

user_simplify_task = [
    """If needed, rewrite the headline and text using terms appropriate for the audience. If not keep it the same.\
    Follow these steps to accomplish this: \
    \n1. Check if the content and language are appropriate for the audience. \
    \n2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n3. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n4. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience is""",
]

simplify_audience = [
    # "a lay audience",
    "people who are not science experts",
]

user_relevance_task = [
    """Rewrite the headline and text to include a statement of how it is relevant for the audience. \
    Follow these steps to accomplish this: \
    \n1. Think about why this might be relevant for the audience in the grand scheme of things.\
    \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, \
    add a sentence to inform the audience. Otherwise, keep it the same. \
    \n3. Modify the content if needed to reduce redundancy. \
    \n4. Check if the content and language are appropriate for the audience. \
    If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ 
    \n5. Return the final version of the headline and text to be shown to the audience. \
    \nIt should be in this format:\
    \n<headline>\n\n<summary>\
    \n\nwhere the summary is in paragraph form.\
    \n6. Remove the backticks. Do not label the headline and summary. ",
    \n\nYour audience consists of""",
]

relevance_audience = [
    "seniors",
    # "people who enjoy sports",
    # "people new to resistance training",
    "people starting an exercise program"
]

## run

In [11]:
# Set parameters
iteration_id = 1
n_choices = 2
pause_per_request=0
chatbot_id = iteration_id
# chatbot_id = 0
save_outputs = False
save = True
# save = False
empty_columns = True
model = 'gpt-3.5-turbo-16k-0613'

# Create initial summaries
chaining_dict = batch_summarize_chain(
    text_dict, folder_path, prep_step, summarize_task, edit_task, chatbot_dict,
    system_role=system_role, model=model,
    n_choices=n_choices, pause_per_request=pause_per_request,
    iteration_id=iteration_id, save_outputs=save_outputs
    )
qna_dict = spreadsheet_columns(
    qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id, save=save
    )

# Create simple summaries
audience = simplify_audience
simple_summaries = prompt_chaining_dict(user_simplify_task, simplify_audience, simple_summaries_dict, 
    chatbot_dict[chatbot_id], iteration_id,
    n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
    )

# Add relevance
relevance = prompt_chaining_dict(user_relevance_task, relevance_audience, relevance_dict, 
    chatbot_dict[chatbot_id], iteration_id, prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, chatbot_id=chatbot_id
    )

# Merge the results
try:
    qna_dict = merge_all_chaining_results(
        chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
        empty_columns=empty_columns, chatbot_id=chatbot_id,
        save_df=save, save_chatbot=save, 
            csv_path=folder_path,
    )
    print(f'\nCompleted merge_all_chaining_results!:)')
except Exception as error:
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    file = f.f_code.co_filename
    print(f'An error occurred on line {lineno} in {file}: {error}')
    print('Unable to merge results')
    if save:
        save_instance_to_dict(chatbot_dict[chatbot_id], ext=None, json_path=folder_path)
        print(f'\nCould not merge; saved Chaining instances as JSON.')

# qna_dict = merge_all_chaining_results2(
#     chatbot_dict, qna_dict, iteration_id=iteration_id, relevance_audiences=2, pivot=True,
#     empty_columns=empty_columns, chatbot_id=chatbot_id,
#         csv_path=folder_path,
# )
# print(f'\nCompleted merge_all_chaining_results!:)')

qna_dict[iteration_id]

**Text #1 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #2 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #3 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
	Done sending request to GPT-3
	...Completed
**Text #4 prompt #1 of 1**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class ins

,A: date,B: folder,C: article_title,D: choice,E: system_role,F: model,G: text,H: prep step,I: summarization task,J: edit task,K: full summarization task,L: summary,M: original summary content rating,N: original summary language rating,O: top summary,P: simple summary choice,Q: audience,R: simplify task,S: full simplify task,T: simple summary,U: simple summary content rating,V: simple summary language rating,W: top simple summary,X: relevance audience 1,Y: add relevance task people starting an exercise program,Z: full relevance task people starting an exercise program,AA: summary: people starting an exercise program,AB: added relevance content rating,AC: added relevance language rating,AD: top added relevance,AE: relevance audience 2,AF: add relevance task seniors,AG: full add relevance task seniors,AH: summary: seniors
0,2023-06-14 1716,2023-06-14 full2,Exercise Snacks A Novel Strategy to Improve Cardiometabolic Health,1,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Exercise Snacks A Novel Strategy to Improve Cardiometabolic Health\n\nABSTRACT\nWe define exerci...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and s...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Exercise Snacks: A Time-Efficient Strategy to Improve Cardiometabolic Health\n\nExercise snacks,...",,,,1,people who are not science experts,"If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...","If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...",Exercise Snacks: A Quick and Easy Way to Improve Your Health\n\nShort bursts of exercise through...,,,,people starting an exercise program,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Exercise Snacks: A Time-Efficient Way to Improve Your Health While Juggling a Busy Schedule\n\nE...,,,,seniors,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Exercise Snacks: A Convenient Way for Seniors to Improve Cardiovascular Health\n\nExercise snack...
1,2023-06-14 1716,2023-06-14 full2,Exercise Snacks A Novel Strategy to Improve Cardiometabolic Health,2,You are a journalist writing content based on science research articles.,gpt-3.5-turbo-16k-0613,Exercise Snacks A Novel Strategy to Improve Cardiometabolic Health\n\nABSTRACT\nWe define exerci...,Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,summarize for a LinkedIn post.,"\nIf applicable, include a brief description of the research participants, such as age and s...",Think about why this might be relevant for the audience in the grand scheme of things. \nIden...,"Exercise Snacks: A Time-Efficient Strategy to Improve Cardiometabolic Health\n\nExercise snacks,...",,,,1,people who are not science experts,"If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...","If needed, rewrite the headline and text using terms appropriate for the audience. If not keep i...",Exercise Snacks: A Quick and Easy Way to Improve Your Health\n\nExercise snacks are short bursts...,,,,people starting an exercise program,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,Rewrite the headline and text to include a statement of how it is relevant for the audience. ...,"Exercise Snacks: A Time-Efficient and Accessible Way to Improve Your Health\n\nExercise snacks, ...",,,,seniors,Rewrite the headline and text to include a stat

# *End of Page*